To calculate the match importance for both teams, the following mathematical model was used. It combines the mathematical necessity of points with the psychological pressure of the tournament stage.

### 1. The Core Variables
First, we define the status of a team before a match starts:
* $P_{current}$: Current points accumulated by the team.
* $M_{played}$: Number of matches already played.
* $M_{total}$: Total league matches (14 for IPL).
* $M_{rem}$: Matches remaining ($M_{total} - M_{played}$).
* $C_{rem}$: Points capacity remaining ($2 \times M_{rem}$).

### 2. Progress Weight (The "Stage" Factor)
This factor scales the importance based on how late it is in the season. A match in the final week carries more "clutch" weight than a match in the first week.
$$W_{progress} = 0.5 + \left( 0.5 \times \frac{M_{played}}{M_{total}} \right)$$
*This scales linearly from $0.5$ (Match 1) to $1.0$ (Match 14).*

### 3. Goal-Based Importance
We calculate pressure for two separate objectives: reaching the **Top 4** and securing a **Top 2** finish.

**Primary Goal: Top 4 (Target = 16 points)**
$$I_{Top4} = 
\begin{cases} 
\frac{16 - P_{current}}{C_{rem}} & \text{if } 0 < (16 - P_{current}) \le C_{rem} \\
0 & \text{otherwise}
\end{cases}$$

**Secondary Goal: Top 2 (Target = 20 points)**
$$I_{Top2} = 
\begin{cases} 
\frac{20 - P_{current}}{C_{rem}} & \text{if } 0 < (20 - P_{current}) \le C_{rem} \\
0 & \text{otherwise}
\end{cases}$$

### 4. Final Importance Calculation
The importance

# Pressure Performance Score (PPS) Formulation 

The **Pressure Performance Score (PPS)** is a specialized metric designed to measure a player's effectiveness under high-stakes conditions. It operationalizes the relationship between raw output and three layers of situational stress: **Contextual** (Match Importance), **Positional** (Entry Point), and **Professional** (Expectation Level).

The calculation for each match is governed by the following formula:

$$PPS_{Match} = \left( \frac{\text{Performance Output} \times \text{Entry Point Stress} \times \text{Stage Magnitude}}{\text{Level Resistance Factor}} \right)$$

---

### **1. Performance Output (The Base Impact)**
This measures the raw contribution of the player in the match, standardized across roles:
* **For Batters:** $Runs \times (Strike Rate / 100)$
* **For Bowlers:** $(Wickets \times 25) + Dots$

### **2. Entry Point Stress (The "Crisis" Factor)**
This captures **when** the player began their task. It identifies "collapse rescuers" by comparing the state of the innings to the time remaining:
* **Formula:** $\frac{Fall \ of \ Wicket + 1}{Over \ of \ wicket + 1}$
* **Logic:** High values (e.g., entering at 15/3 in Over 2) significantly multiply the score. Low values (e.g., entering at 180/3 in Over 19) result in a lower multiplier.

### **3. Stage Magnitude (The "Big Stage" Factor)**
Utilizes the pre-calculated numerical importance values from the `league.csv` file for the specific team.
* **Formula:** $e^{(Importance_{Team} / 5)}$
* **Logic:** The exponential $e$ ensures that high-stakes matches (Playoffs/Finals) carry exponentially more weight than standard league games.

### **4. Level Resistance Factor (The "Expectation" Filter)**
This adjusts the score based on the player's status in `player_details.csv`. Higher status players have a higher "resistance" because they are expected to handle pressure as a baseline.
* **Uncapped:** Assigned **0.5** (Doubles the score to reward breakthrough performances).
* **Capped:** Assigned **1.0** (Standard international expectation).
* **Elite:** Assigned **1.5** (Highest resistance; elite players must produce massive impact to move their score significantly).

### **5. Historical Reputation (EMA)**
To ensure the score reflects long-term reliability, an **Exponential Moving Average (EMA)** is applied across all previous match data. This weights recent "clutch" performances more heavily while preserving a player’s big-game reputation.

### **6. Professional Selection Baseline & Sigmoid Scaling**
Because all 247+ players are professionals, the model assumes a **Baseline of 30** (representing the standard pressure-handling capability required to be selected for a squad). The remaining 70 points are distributed via a Sigmoid function based on performance.

* **Formula:** $30 + \left( \frac{70}{1 + e^{-Z}} \right)$
* **Result:** * **30 Points:** The "Selection Floor" for any player in the roster with no match data or 0 impact.
    * **50-60 Points:** Average professional performance under pressure.
    * **90+ Points:** Elite "Clutch" performers who consistently win high-importance matches.

# Consistency Score (CS) Formulation (Form-Adjusted)

The model operationalizes consistency as the stability of a player's **Form**, rather than their historical average alone. It identifies "Reliable Anchors" by rewarding those who maintain a steady output in their most recent appearances.

The final score is mapped to a professional scale using the following formula:

$$CS = 30 + \left( \frac{70}{1 + e^{-Z_{CS}}} \right)$$

Where the $Z$-score is derived from the **Exponential Stability Index ($ESI$):**

$$ESI = \left( \frac{EMA(Output)}{\text{Volatility Index} \times \text{Level Resistance Factor}} \right)$$

---

### **1. Performance Form ($EMA$)**
Instead of a simple arithmetic mean, the model utilizes the **Exponential Moving Average (EMA)** of match-level output to prioritize "Current Form."
* **Calculation:** $Output_{Match} = (Runs \times \frac{SR}{100})$ for Batters or $(Wkts \times 25 + Dots)$ for Bowlers.
* **Logic:** The EMA applies a decay factor that gives significantly higher weight to recent matches, ensuring the score reflects the player's current reliability rather than legacy performances.

### **2. Volatility Index (The "Form Variance")**
This measures how much the player’s recent scores fluctuate around their current Form.
* **Formula:** $1 + \left( \frac{\text{Standard Deviation of last 5 matches}}{EMA + 1} \right)$
* **Logic:** A player whose last five scores are tightly clustered (e.g., [45, 48, 44, 46, 45]) will have a low Volatility Index, resulting in an extremely high Consistency Score because their "Form" is locked in.

### **3. Level Resistance Factor (Expectation Tier)**
Consistent performance is weighted differently based on the player's professional tier to account for the "Expectation Burden":
* **Uncapped (0.5):** Rewards emerging players who show a stable baseline early in their careers.
* **Capped (1.0):** The standard professional benchmark for reliability.
* **Elite (1.5):** The "Elite Burden"—these players are expected to be the most reliable, so any significant dip in form results in a heavier penalty.

### **4. Professional Selection Baseline**
* **The 30-Point Floor:** Every one of the 247+ players starts at a base of 30 points.
* **Logic:** This represents the inherent "form" and technical standard required to be selected for a professional squad in the first place. Inactive players or those with insufficient data remain at this selection baseline.

# Consistency Score (CS) Formulation (Form-Adjusted)

The model operationalizes consistency as the stability of a player's **Form**, rather than their historical average alone. It identifies "Reliable Anchors" by rewarding those who maintain a steady output in their most recent appearances.

The final score is mapped to a professional scale using the following formula:

$$CS = 30 + \left( \frac{70}{1 + e^{-Z_{CS}}} \right)$$

Where the $Z$-score is derived from the **Exponential Stability Index ($ESI$):**

$$ESI = \left( \frac{EMA(Output)}{\text{Volatility Index} \times \text{Level Resistance Factor}} \right)$$

---

### **1. Performance Form ($EMA$)**
Instead of a simple arithmetic mean, the model utilizes the **Exponential Moving Average (EMA)** of match-level output to prioritize "Current Form."
* **Calculation:** $Output_{Match} = (Runs \times \frac{SR}{100})$ for Batters or $(Wkts \times 25 + Dots)$ for Bowlers.
* **Logic:** The EMA applies a decay factor that gives significantly higher weight to recent matches, ensuring the score reflects the player's current reliability rather than legacy performances.

### **2. Volatility Index (The "Form Variance")**
This measures how much the player’s recent scores fluctuate around their current Form.
* **Formula:** $1 + \left( \frac{\text{Standard Deviation of last 5 matches}}{EMA + 1} \right)$
* **Logic:** A player whose last five scores are tightly clustered (e.g., [45, 48, 44, 46, 45]) will have a low Volatility Index, resulting in an extremely high Consistency Score because their "Form" is locked in.

### **3. Level Resistance Factor (Expectation Tier)**
Consistent performance is weighted differently based on the player's professional tier to account for the "Expectation Burden":
* **Uncapped (0.5):** Rewards emerging players who show a stable baseline early in their careers.
* **Capped (1.0):** The standard professional benchmark for reliability.
* **Elite (1.5):** The "Elite Burden"—these players are expected to be the most reliable, so any significant dip in form results in a heavier penalty.

### **4. Professional Selection Baseline**
* **The 30-Point Floor:** Every one of the 247+ players starts at a base of 30 points.
* **Logic:** This represents the inherent "form" and technical standard required to be selected for a professional squad in the first place. Inactive players or those with insufficient data remain at this selection baseline.

# Phased Performance: Bowler Specifics

While Batters are measured on scoring speed, **Bowlers** are evaluated on their ability to create pressure and take wickets. The model differentiates between "Strike Bowlers" and "Containment Specialists."

### 1. Bowling Impact Component (RSI)
For the Role-Specific Impact, a bowler's raw output is calculated by balancing aggression (Wickets) with control (Dots/Economy).

| Variable | Weight | Logic |
| :--- | :--- | :--- |
| **Wickets** | **+25 pts** | The highest value action for a bowler. |
| **Dot Balls** | **+1 pt** | Measures the ability to build pressure and stall the run rate. |
| **Economy Rate** | **-5 pts** | A penalty multiplier; high leakage reduces the overall impact score. |

**The Bowler's RSI Formula:**
$$Raw \ Impact_{Bowl} = \frac{(Wkts \times 25) + Dots - (Econ \times 5)}{\text{Level Resistance Factor}}$$



---

### 2. Bowling Phase Metrics (PhasePS)
A bowler's value changes depending on when they bowl. The **PhasePS** rewards different skills for the three segments:

#### **A. Opening Phase (0-6 Overs)**
* **Primary Goal:** Early breakthroughs and containment.
* **Logic:** Heavily weights **Wickets** and **Low Economy**. A bowler who can swing the new ball and keep the score under 6 an over in the Powerplay ranks highest.

#### **B. Middle Phase (7-15 Overs)**
* **Primary Goal:** Building pressure and breaking partnerships.
* **Logic:** Weights **Dot Ball %** and **Wickets**. This rewards spinners and "enforcer" seamers who dry up the runs and force mistakes.

#### **C. Death Phase (16-20 Overs)**
* **Primary Goal:** Preventing the "Surge" and cleaning up the tail.
* **Logic:** Heavily weights **Wickets** and **Boundary Prevention**. Bowlers who nail Yorkers and slower balls under extreme pressure are rewarded here.

---

### 3. The Bowler's Versatility Bonus (+10 Points)
The **Additive Versatility Bonus** specifically rewards bowlers who can "cross over" between these specialized roles.

* **The "All-Phase Enforcer":** If a designated **Death Specialist** (high career activity in overs 16-20) produces an **Opening_Score > 65**, they receive the **+10 point bonus**.
* **Logic:** This identifies rare bowlers who possess both the finesse for the new ball and the tactical nerves for the old ball.



---

### 4. Professional Tiering for Bowlers
The **Level Resistance Factor** ensures that an **Elite** bowler (e.g., a world-class international) is held to a higher standard of consistency than an **Uncapped** bowler.
* **Elite Bowler:** Must maintain low economy and high strike rates across multiple matches to sustain a score above 80.
* **Uncapped Bowler:** A single "3-wicket haul" or a maiden over in the Powerplay can significantly boost their score due to the 0.5 resistance multiplier.

# Team Contribution Score (TCS) Formulation

The **Team Contribution Score (TCS)** measures a player's relative impact on the team's success during critical tournament phases. It quantifies how much a player "carries" the team when the match importance is high.

### 1. Relative Contribution (The Base)
The score begins by calculating the player's share of the team's total output for a specific match:
* **Batting Contribution ($Rel_{Runs}$):** `Player Runs / (Team Total Runs + 1e-9)`.
* **Bowling Contribution ($Rel_{Wkts}$):** `Player Wickets / (Team Total Wickets + 1e-9)`.

### 2. Situational & Result Multipliers
* **Entry Point Stress (EPW):** `(Fall of Wicket + 1) / (Over of Wicket + 1)` for batters.
* **Innings Context:** A **1.2x multiplier** for bowlers in the 2nd innings (defending a total).
* **Win Factor:** A **1.2x multiplier** for a team win; **0.8x** for a loss.

### 3. The Exponential Stakes Factor
The contribution is scaled by the mathematical importance of the match:
$$Raw \ TCS = (Rel \ Output \times Win \ Factor \times EPW \times e^{\frac{Match \ Importance}{5}})$$

### 4. Professional Calibration & Penalty
* **Level Resistance:** The score is divided by the player's tier: Uncapped (**0.5**), Capped (**1.0**), or Elite (**1.5**).
* **Big-Match Penalty:** A **-15 point penalty** is applied if the player's average in high-importance matches ($Imp > 7$) is **< 70%** of their season average.

### 5. Final Scaling
$$TCS = 30 + \left( \frac{70}{1 + e^{-Z_{TCS}}} \right)$$
*This maps the impact to a 30–100 scale, where 30 is the professional selection baseline.*

# Fielding Impact Score (FIS) Formulation

The **Fielding Impact Score (FIS)** operationalizes "Clutch Fielding" by rewarding technical difficulty and the situational value of the wicket taken.

### 1. Wicket Importance (Target Value)
Catches are weighted by the quality of the dismissed batsman:
* **Batting Value:** Derived from the batsman's match performance ($Runs \times \frac{SR}{100}$).
* **Wicket Multiplier:** Scaled between **0.5 and 2.5** based on the batsman's impact.

### 2. Catch Difficulty ($CD$)
Technical difficulty multipliers:
* **Hard:** 3.5x
* **Medium:** 1.8x
* **Easy:** 1.0x

### 3. Match Stakes
The fielding action is amplified by the importance of the match to the team:
$$e^{\frac{Match \ Importance}{5}}$$

### 4. Raw Impact Formula
Every catch, stumping, or run-out is calculated as:
$$Raw \ Impact = CD \times Wicket \ Multiplier \times e^{\frac{Match \ Importance}{5}}$$

### 5. Level Resistance & Final Scaling
* **Level Resistance:** The raw impact is normalized by the player tier (Uncapped: **0.5**, Capped: **1.0**, Elite: **1.5**).
* **The 30-Point Floor:** Every player starts at a baseline of 30.
* **Sigmoid Mapping:** The adjusted impact is converted into a final 30–100 score via a Z-score distribution.

# Role-Specific Impact (RSI) Formulation

The **RSI Score** identifies the "Statistical Value" of a player by calculating a match-level impact score and scaling it to a professional benchmark.

### **1. Performance Components (Raw Impact)**
The model calculates a base impact value depending on the player's actions in a match:

* **Batting Impact ($Impact_{Bat}$):**
  $$(Runs \times \frac{Strike Rate}{100}) + (Sixes \times 2) + Fours$$
  *This rewards scoring volume, scoring speed, and boundary-hitting capability.*

* **Bowling Impact ($Impact_{Bowl}$):**
  $$(Wickets \times 25) + Dot Balls - (Economy \times 5)$$
  *This rewards wicket-taking and pressure-building (dots) while penalizing run leakage.*

### **2. Weighted Role Logic**
The model applies different weights to the raw impact based on the player's designated `Role`:
* **Batter / Wicket-Keeper:** Uses 100% of the Batting Impact.
* **Bowler:** Uses 100% of the Bowling Impact.
* **All-Rounder:** Uses a balanced weight of **60% Batting + 60% Bowling**.

### **3. Professional Tiering (Level Resistance)**
The raw RSI is divided by a **Level Resistance Factor** to normalize expectations:
* **Uncapped:** 0.5 (Breakthrough multiplier).
* **Capped:** 1.0 (Standard professional benchmark).
* **Elite:** 1.5 (High-performance burden).

### **4. Final Scaling**
$$RSI = 30 + \left( \frac{70}{1 + e^{-Z_{RSI}}} \right)$$
*This maps the impact to a 30–100 scale, where 30 is the professional selection baseline.*